In [7]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import numpy as np
import pandas as pd
import tensorflow as tf
from numpy import array
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble


#주어진 문장을 '단어'로 토큰화 하기
import gensim.models
import matplotlib.pyplot as plt
import seaborn as sns

#케라스의 텍스트 전처리와 관련한 함수중 text_to_word_sequence 함수를 불러 옵니다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence
 

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [8]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [9]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [26]:
train['keyword'].value_counts()

fatalities               45
deluge                   42
armageddon               42
harm                     41
body%20bags              41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [31]:
print(train.groupby('keyword')['target'].mean().sort_values())

keyword
aftershock     0.000000
body%20bags    0.024390
ruin           0.027027
blazing        0.029412
body%20bag     0.030303
                 ...   
typhoon        0.973684
outbreak       0.975000
debris         1.000000
wreckage       1.000000
derailment     1.000000
Name: target, Length: 221, dtype: float64


In [34]:
print(train.groupby('keyword')['target'].mean())

keyword
ablaze                 0.361111
accident               0.685714
aftershock             0.000000
airplane%20accident    0.857143
ambulance              0.526316
                         ...   
wounded                0.702703
wounds                 0.303030
wreck                  0.189189
wreckage               1.000000
wrecked                0.076923
Name: target, Length: 221, dtype: float64


In [ ]:
print('train[\'text\']의 결측치 개수', train['text'].isnull().sum())
print('test[\'text\']의 결측치 개수', test['text'].isnull().sum())
print('train[\'keyword\']의 결측치 개수', train['keyword'].isnull().sum())
print('test[\'keyword\']의 결측치 개수', test['keyword'].isnull().sum())
print('train[\'location\']의 결측치 개수', train['location'].isnull().sum())
print('test[\'location\']의 결측치 개수', test['location'].isnull().sum())

In [ ]:
corpus=train['text'].values

In [ ]:
corpus=np.concatenate((corpus,test['text'].values), axis=0)

In [ ]:
corpus.size

In [ ]:
temp=[]
for tem in corpus:
    temp.append(re.sub(r'[^a-zA-Z0-9#@ ]',' ',tem).lower())

In [ ]:
corpus=np.array(temp)

In [ ]:
corpus

In [ ]:
lm = WordNetLemmatizer()


In [ ]:
temp=[]
for tem in corpus:
    temp.append(tem.split())

In [ ]:
temp2=[]
for temp_ in temp:
    temp2.append([lm.lemmatize(w, pos="v") for w in temp_])

In [ ]:
corpus=[]
for tem2 in temp2:
    corpus.append(' '.join(tem2))

In [ ]:
corpus

In [ ]:
# Word2Vec 생성
sentences = []
#reviews = clean_train_reviews
for review in corpus:
    sentences.append(review.split())   # [['stuff',  'going',  'moment',  'mj',  'started',  'listening',  'music', ... ], ...] #25000개의 리스트

print(len(sentences))  #25000

from gensim.models import word2vec
word2vec_model = word2vec.Word2Vec(sentences, workers=1, size=10, min_count = 2, window = 10)

In [ ]:
temp=train['keyword'].values
aa=[]
for tem in temp:
    if tem is not np.nan:
        aa.append(re.sub(r'[^a-zA-Z]',' ',tem).lower().split())
#    else:
 #       aa.append('')
temp=[]
for a in aa:
    temp.append([lm.lemmatize(w,pos='v') for w in a])

mean = np.zeros((10), dtype=np.float32) # keyword의 평균값구하는 식
time = 0
for a in temp:
    time = time +1
    for b in a:
        mean = mean + word2vec_model[b]
        time = time +1
    
mean = mean / time 

In [ ]:
mean #keyword의 평균값

In [ ]:
temp=train['keyword'].values
aa=[]
for tem in temp:
    if tem is not np.nan:
        aa.append(re.sub(r'[^a-zA-Z]',' ',tem).lower().split())
    else:
        aa.append('')
temp=[]
for a in aa:
    temp.append([lm.lemmatize(w,pos='v') for w in a])

vec = np.empty(0) # keyword의 평균값구하는 식 각 키워드에 대해 벡터화 시키는 코드
print(vec)
time = 0
for a in temp:
    time = 0
    numpy = np.zeros((10), dtype=np.float32)
    if not a:
        time=1
        numpy = numpy + mean
    else:    
        for b in a:
            time=time+1
            if b in word2vec_model:
                numpy = numpy + word2vec_model[b] # 값이 있으면 그냥 더하기
            else:
                numpy = numpy + mean #평균값 더하기
        numpy = numpy / time # 여러 단어의 keyword로 구성되었다면 평균으로 나누어 주기
    vec=np.concatenate((vec,numpy), axis=0)

In [ ]:
vec=vec.reshape(-1,10)

In [ ]:
vec.shape

In [ ]:
#텍스트만을 독립변수로 했을 때의 적중률이 상당히 낮게 나왔음
#텍스트의 성질을 분석하여 int형으로 할당한 후 신경망 계산
#분산표현을 사용하는 곳은 keyword 칼럼
train['ht_number'] = train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
test['ht_number'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

In [ ]:

train['email_number'] = train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
test['email_number'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

In [ ]:
train['text_length'] = train['text'].apply(lambda x: len(x))
test['text_length'] = test['text'].apply(lambda x: len(x))

In [ ]:
train = train.drop('id',axis=1)
train = train.drop('location',axis=1)



In [ ]:
print(train['ht_number'].values.size)
print(train['email_number'].values.size)
print(train['text_length'].values.size)
print(vec.shape)
print(vec[0])
vec= np.append(vec,train['ht_number'].values.reshape(-1,1),axis=1)
vec= np.append(vec,train['email_number'].values.reshape(-1,1),axis=1)
vec= np.append(vec,train['text_length'].values.reshape(-1,1),axis=1)

In [ ]:
vec.shape

In [ ]:
#vec를 keyword의 벡터화시키고, email 개수, 해쉬태그 개수, 문장의 길이 데이터를 numpy 배열에 추가

In [ ]:
train = train.values

In [ ]:
train

In [ ]:
Y_train = train[:,2]

In [ ]:
X_train = vec

In [ ]:
X_train.shape

In [ ]:
Y_train

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X_train, Y_train, test_size=0.3, random_state=23)

In [ ]:
print(X_train.shape)
print(y_train)

In [ ]:

wineRFModel = ensemble.RandomForestRegressor(n_estimators=100,
                    max_depth=50,oob_score=False, random_state=31)
wineRFModel.fit(X_train, y_train)

In [ ]:
prediction = wineRFModel.predict(X_test)

In [ ]:
collect=np.sum(np.around(prediction)==y_test)

In [ ]:
collect/len(y_test)

In [ ]:
temp=test['keyword'].values
aa=[]
for tem in temp:
    if tem is not np.nan:
        aa.append(re.sub(r'[^a-zA-Z]',' ',tem).lower().split())
    else:
        aa.append('')
temp=[]
for a in aa:
    temp.append([lm.lemmatize(w,pos='v') for w in a])

vec = np.empty(0) # keyword의 평균값구하는 식 각 키워드에 대해 벡터화 시키는 코드
print(vec)
time = 0
for a in temp:
    time = 0
    numpy = np.zeros((10), dtype=np.float32)
    if not a:
        time=1
        numpy = numpy + mean
    else:    
        for b in a:
            time=time+1
            if b in word2vec_model:
                numpy = numpy + word2vec_model[b] # 값이 있으면 그냥 더하기
            else:
                numpy = numpy + mean #평균값 더하기
        numpy = numpy / time # 여러 단어의 keyword로 구성되었다면 평균으로 나누어 주기
    vec=np.concatenate((vec,numpy), axis=0)
vec=vec.reshape(-1,10)

vec= np.append(vec,test['ht_number'].values.reshape(-1,1),axis=1)
vec= np.append(vec,test['email_number'].values.reshape(-1,1),axis=1)
vec= np.append(vec,test['text_length'].values.reshape(-1,1),axis=1)

In [ ]:
answer=np.round(wineRFModel.predict(vec)).astype(int)

In [ ]:
submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
submission['target'] = answer

In [ ]:

submission.to_csv('submission.csv', index=False, header=True)

In [ ]:
answer